In [6]:
import enlighten

manager = enlighten.get_manager()

# Standard bar format
std_bar_format = u'{desc}{desc_pad}{percentage:3.0f}%|{bar}| ' + \
                 u'{count:{len_total}d}/{total:d} ' + \
                 u'[{elapsed}<{eta}, {rate:.2f}{unit_pad}{unit}/s]'

# Red text
bar_format = manager.term.red(std_bar_format)

# Red on white background
bar_format = manager.term.red_on_white(std_bar_format)

# X11 colors
bar_format = manager.term.peru_on_seagreen(std_bar_format)

# RBG text
bar_format = manager.term.color_rgb(2, 5, 128)(std_bar_format)

# RBG background
bar_format = manager.term.on_color_rgb(255, 190, 195)(std_bar_format)

# RGB text and background
bar_format = manager.term.on_color_rgb(255, 190, 195)(std_bar_format)
bar_format = manager.term.color_rgb(2, 5, 128)(bar_format)

# Apply color to select parts
bar_format = manager.term.red(u'{desc}') + u'{desc_pad}' + \
             manager.term.blue(u'{percentage:3.0f}%') + u'|{bar}|'

# Apply to counter
ticks = manager.counter(total=100, desc='Ticks', unit='ticks', bar_format=bar_format)

In [28]:


manager = enlighten.get_manager()
ticks = manager.counter(total=100, desc='Ticks', unit='ticks',color="blue")
tocks = manager.counter(total=20, desc='Tocks', unit='tocks')

for num in range(100):
    time.sleep(0.1)  # Simulate work
    print(num)
    #ticks.desc = 'Tocks' + str(num)
    ticks.update()
    for i in range(20):
        time.sleep(0.01)  # Simulate work
        tocks.update()

manager.stop()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [39]:
from multiprocessing import Process, Queue
import random
import time

import enlighten


WORKERS = 10

FILES = (100, 200)
FILE_TIME = (0.01, 0.05)


def process_files(queue, count):
    """
    Simple child processor

    Sleeps for a random interval and pushes the current count onto the queue
    """

    for num in range(1, count + 1):
        time.sleep(random.uniform(*FILE_TIME))  # Random processing time
        queue.put(num)



manager = enlighten.get_manager()
systems = 20
started = 0
active = {}
pb_started = manager.counter(total=systems, desc='Systems:', unit='systems', color='yellow')
pb_finished = pb_started.add_subcounter('green', all_fields=True)

# Loop until all systems finish
while systems > started or active:

    # If there are free workers and tasks left to run, start them
    if systems > started and len(active) < WORKERS:
        queue = Queue()
        files = random.randint(*FILES)
        process = Process(target=process_files, args=(queue, files))
        started += 1
        counter = manager.counter(total=files, desc='  System %d:' % started,
                                  unit='files', leave=False)
        process.start()
        pb_started.update()
        active[started] = (process, queue, counter)

    # Iterate through active subprocesses
    for system in tuple(active.keys()):
        process, queue, counter = active[system]
        alive = process.is_alive()

        # Latest count is the last one on the queue
        count = None
        while not queue.empty():
            count = queue.get()

        # Update counter. We do it manually because we have the number not the increment
        if count is not None:
            counter.count = count
            # If no sleep is used in loop use counter.update(0) instead
            counter.refresh()

        # Remove any finished subprocesses and update progress bar
        # If this was real code you could check for failures
        if not alive:
            counter.close()
            print('Processed %d files on System %d' % (counter.total, system))
            del active[system]
            pb_finished.update_from(pb_started)

    # Sleep for 1/10 of a second to reduce load
    time.sleep(0.1)



Processed 109 files on System 1
Processed 138 files on System 2
Processed 137 files on System 3
Processed 188 files on System 4
Processed 151 files on System 5
Processed 164 files on System 6
Processed 114 files on System 7
Processed 190 files on System 8
Processed 152 files on System 9
Processed 159 files on System 10
Processed 174 files on System 11
Processed 158 files on System 12
Processed 133 files on System 13
Processed 153 files on System 14
Processed 185 files on System 15
Processed 114 files on System 16
Processed 188 files on System 17
Processed 165 files on System 18
Processed 127 files on System 19
Processed 110 files on System 20
